In [20]:
import sqlite3
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Récupération des données

In [21]:
conn = sqlite3.connect('../../db.sqlite3')

query_pizzas = """
SELECT pizza_id, name, size, ingredients
FROM menu;
"""
pizzas_df = pd.read_sql_query(query_pizzas, conn)

query_orders = """
SELECT client_id, pizza_id, order_date
FROM orders;
"""
orders_df = pd.read_sql_query(query_orders, conn)

conn.close()


Application de TF-IDF

In [22]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(pizzas_df['ingredients'])


cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

Fonction de recommandation par client

In [23]:
def recommend_pizza_for_client(client_id, orders_df, pizzas_df, cosine_sim, num_recommendations=1):
    # Récupérer les pizzas commandées par le client
    client_orders = orders_df[orders_df['client_id'] == client_id]
    ordered_pizza_ids = client_orders['pizza_id'].unique()

    # Récupérer les indices des pizzas dans le DataFrame
    pizza_indices = pizzas_df[pizzas_df['pizza_id'].isin(ordered_pizza_ids)].index

    # Calculer la similarité moyenne des pizzas commandées par le client
    similar_scores = cosine_sim[pizza_indices].mean(axis=0)

    # Trier les pizzas par similarité décroissante, en excluant les pizzas déjà commandées
    pizza_scores = list(enumerate(similar_scores))
    pizza_scores = [(i, score) for i, score in pizza_scores if pizzas_df.iloc[i]['pizza_id'] not in ordered_pizza_ids]
    pizza_scores = sorted(pizza_scores, key=lambda x: x[1], reverse=True)

    # Récupérer les 'num_recommendations' pizzas les plus similaires
    recommended_pizzas = []
    total_similarity = 0
    recommended_pizza_names = set()  # Utilisation d'un set pour s'assurer que les noms sont uniques

    for i in range(len(pizza_scores)):
        recommended_pizza_idx = pizza_scores[i][0]
        recommended_pizza = pizzas_df.iloc[recommended_pizza_idx]
        
        # Vérifier si le nom de la pizza est déjà recommandé
        if recommended_pizza['name'] not in recommended_pizza_names:
            recommended_pizzas.append((recommended_pizza['name'], recommended_pizza['pizza_id'], pizza_scores[i][1]))
            recommended_pizza_names.add(recommended_pizza['name'])
            total_similarity += pizza_scores[i][1]
        
        # Arrêter une fois que nous avons atteint le nombre de recommandations souhaité
        if len(recommended_pizzas) == num_recommendations:
            break

    # Calculer le score moyen de similarité pour les recommandations
    average_similarity = total_similarity / len(recommended_pizzas) if recommended_pizzas else 0

    # Calcul des métriques de précision et de couverture
    ordered_pizza_names = pizzas_df[pizzas_df['pizza_id'].isin(ordered_pizza_ids)]['name'].unique()
    recommended_pizza_names_in_history = recommended_pizza_names.intersection(ordered_pizza_names)

    # Précision : combien de pizzas recommandées (par nom) sont déjà dans l'historique
    precision = len(recommended_pizza_names_in_history) / len(recommended_pizza_names) if recommended_pizza_names else 0

    # Couverture : combien de pizzas recommandées (par nom) sont nouvelles (non commandées avant)
    coverage = 1 - precision  # Couverture est simplement le complément de la précision

    return recommended_pizzas, average_similarity, precision, coverage


Recommandations pour tous les clients

In [24]:
def recommend_for_all_clients(orders_df, pizzas_df, cosine_sim, num_recommendations=1):
    recommendations = []
    metrics = []
    global_similarity = 0
    global_precision = 0
    global_coverage = 0
    client_ids = orders_df['client_id'].unique()

    for client_id in client_ids:
        # Obtenez les recommandations et les métriques pour chaque client
        recommended_pizzas, avg_similarity, precision, coverage = recommend_pizza_for_client(
            client_id, orders_df, pizzas_df, cosine_sim, num_recommendations
        )
        
        # Ajoutez les résultats pour chaque client
        recommendations.append((client_id, recommended_pizzas))
        metrics.append((client_id, avg_similarity, precision, coverage))
        
        # Ajoutez aux métriques globales
        global_similarity += avg_similarity
        global_precision += precision
        global_coverage += coverage

    # Calcul des moyennes globales
    num_clients = len(client_ids)
    global_similarity /= num_clients
    global_precision /= num_clients
    global_coverage /= num_clients

    return recommendations, metrics, global_similarity, global_precision, global_coverage

# Exemple : Recommander des pizzas pour tous les clients (par exemple recommander 3 pizzas par client)
num_recommendations = 3
recommendations, metrics, global_similarity, global_precision, global_coverage = recommend_for_all_clients(
    orders_df, pizzas_df, cosine_sim, num_recommendations
)

# Afficher les recommandations et les métriques par client
for client_id, recommended_pizzas in recommendations:
    print(f"Recommandations pour le client {client_id}:")
    for pizza_name, pizza_id, score in recommended_pizzas:
        print(f"  Pizza recommandée: {pizza_name} (ID: {pizza_id}), Score de similarité: {score:.4f}")

Recommandations pour le client 1:
  Pizza recommandée: The Spinach Supreme Pizza (ID: spinach_supr_s), Score de similarité: 0.3721
  Pizza recommandée: The Spinach and Feta Pizza (ID: spinach_fet_l), Score de similarité: 0.3663
  Pizza recommandée: The Mediterranean Pizza (ID: mediterraneo_l), Score de similarité: 0.3188
Recommandations pour le client 2:
  Pizza recommandée: The Greek Pizza (ID: the_greek_xxl), Score de similarité: 0.7007
  Pizza recommandée: The Spinach Supreme Pizza (ID: spinach_supr_s), Score de similarité: 0.3091
  Pizza recommandée: The Spinach and Feta Pizza (ID: spinach_fet_l), Score de similarité: 0.3083
Recommandations pour le client 3:
  Pizza recommandée: The Thai Chicken Pizza (ID: thai_ckn_m), Score de similarité: 0.3455
  Pizza recommandée: The Spinach Supreme Pizza (ID: spinach_supr_s), Score de similarité: 0.2490
  Pizza recommandée: The Mediterranean Pizza (ID: mediterraneo_l), Score de similarité: 0.2399
Recommandations pour le client 4:
  Pizza recom

Analyse des performances

In [25]:
# Afficher les métriques par client
print("\nMétriques par client:")
for client_id, avg_similarity, precision, coverage in metrics:
    print(f"  Client {client_id} - Similarité moyenne: {avg_similarity:.4f}, Précision: {precision:.4f}, Couverture: {coverage:.4f}")


Métriques par client:
  Client 1 - Similarité moyenne: 0.3524, Précision: 0.6667, Couverture: 0.3333
  Client 2 - Similarité moyenne: 0.4394, Précision: 0.3333, Couverture: 0.6667
  Client 3 - Similarité moyenne: 0.2781, Précision: 0.6667, Couverture: 0.3333
  Client 4 - Similarité moyenne: 0.3100, Précision: 1.0000, Couverture: 0.0000
  Client 5 - Similarité moyenne: 0.5816, Précision: 0.6667, Couverture: 0.3333
  Client 6 - Similarité moyenne: 0.2812, Précision: 0.0000, Couverture: 1.0000
  Client 7 - Similarité moyenne: 0.3111, Précision: 1.0000, Couverture: 0.0000
  Client 8 - Similarité moyenne: 0.2794, Précision: 1.0000, Couverture: 0.0000
  Client 9 - Similarité moyenne: 0.3898, Précision: 1.0000, Couverture: 0.0000
  Client 10 - Similarité moyenne: 0.2876, Précision: 1.0000, Couverture: 0.0000
  Client 11 - Similarité moyenne: 0.3277, Précision: 1.0000, Couverture: 0.0000
  Client 12 - Similarité moyenne: 0.5513, Précision: 0.6667, Couverture: 0.3333
  Client 13 - Similarité m

In [26]:
# Afficher les métriques globales
print("\nMétriques globales :")
print(f"  Similarité moyenne globale: {global_similarity:.4f}")
print(f"  Précision moyenne globale: {global_precision:.4f}")
print(f"  Couverture moyenne globale: {global_coverage:.4f}")


Métriques globales :
  Similarité moyenne globale: 0.4030
  Précision moyenne globale: 0.7039
  Couverture moyenne globale: 0.2961
